# Binary data with lazy loading

**Author**: Aureliana Barghini ([B-Open](https://www.bopen.eu/))

If you want to make your backend effective with big datasets, then you should
support lazy loading. <br/>
For doing that you need:
-  Implement `_raw_indexing_method` for reading blocks form disk<br/><br/>
- Implement some glue code to make it work with Xarray:<br/><br/>
    - put your `_raw_indexing_method` in a `BackendArray` subclass <br/><br/>
    - replace the `numpy.ndarray` inside your **dataset** with your subclass of `BackendArray`
    
    
<br/>

## Create sample files

In [ ]:
import os

import dask
import numpy as np
import xarray as xr

In [ ]:
arr = np.arange(30000000, dtype=np.int64)
with open("foo.bin", "w") as f:
    arr.tofile(f)

arr = np.arange(30000000, dtype=np.float64)
with open("foo_float.bin", "w") as f:
    arr.tofile(f)

## BinaryBackendArray

The BackendArray subclass shall implement the following method and attributes:

- `_raw_indexing_method` method, supporting **item selection** and **slicing**

- `__getitem__` that wraps `_raw_indexing_method` with an xarray helper function `explicit_indexing_adapter` (threadsafe)

- `shape` attribute

- `dtype` attribute.



In [ ]:
class BinaryBackendArray(xr.backends.BackendArray):
    def __init__(
        self,
        filename_or_obj,
        shape,
        dtype,
        lock,
    ):
        self.filename_or_obj = filename_or_obj
        self.shape = shape
        self.dtype = dtype
        self.lock = lock

    def __getitem__(self, key: tuple):
        return xr.core.indexing.explicit_indexing_adapter(
            key,
            self.shape,
            xr.core.indexing.IndexingSupport.BASIC,
            self._raw_indexing_method,
        )

    def _raw_indexing_method(self, key: tuple):
        key0 = key[0]
        size = np.dtype(self.dtype).itemsize

        if isinstance(key0, slice):
            start = key0.start or 0
            stop = key0.stop or self.shape[0]
            offset = size * start
            count = stop - start
        else:
            offset = size * key0
            count = 1

        with self.lock, open(self.filename_or_obj) as f:
            arr = np.fromfile(f, np.int64, offset=offset, count=count)

        if isinstance(key, int):
            arr = arr.squeeze()

        return arr

## BinaryBackend Entrypoint

In [ ]:
class BinaryBackend(xr.backends.BackendEntrypoint):
    def open_dataset(self, filename_or_obj, *, drop_variables=None, dtype=np.int64):
        size = np.dtype(dtype).itemsize
        shape = os.stat(filename_or_obj).st_size // size

        backend_array = BinaryBackendArray(
            filename_or_obj=filename_or_obj,
            shape=(shape,),
            dtype=dtype,
            lock=dask.utils.SerializableLock(),
        )
        data = xr.core.indexing.LazilyIndexedArray(backend_array)

        var = xr.Variable(dims=("x"), data=data)
        return xr.Dataset({"foo": var})


## Reduced memory usage with dask

In [ ]:
arr = xr.open_dataarray("foo.bin", engine=BinaryBackend, chunks=10000)
arr

In [ ]:
arr.mean()

In [ ]:
arr.sel(x=slice(0, 10))

In [ ]:
arr.sel(x=slice(0, 10)).compute()

In [ ]:
arr.load()